# Ingestão de Dados ELT
> Conjunto de dados de previsão de risco de doenças cardiovasculares

## Verificando pastas criadas

In [0]:
display(dbutils.fs)

In [0]:
display(dbutils.fs.ls("/"))

## Extraindo dados/Realizando a leitura

In [0]:
df = spark.read.format("csv").option("header", True).load("/tmp/cardiovascular-diseases-risk.csv")

In [0]:
df.display()

## Verificando valores distintos

In [0]:
df.select("General_Health").distinct().display()

## Alterando nome da coluna (o Delta Table não aceita caracteres especiais em nome de colunas)

In [0]:
df.printSchema()

In [0]:
df = df.withColumnRenamed("Height_(cm)", "Height_cm").withColumnRenamed("Weight_(kg)", "Weight_kg")

## Realizando o load/Armazenamento de dados com configurações especificas

In [0]:
df.write.format("delta").mode("overwrite").option("mergeSchema", "true").partitionBy("General_Health").save("/hospital/rw/cardiovascular_diseases/")

# Criar database e tabela delta location storage

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS db_hospital

In [0]:
%sql
CREATE TABLE IF NOT EXISTS db_hospital.cardiovascular_diseases LOCATION "/hospital/rw/cardiovascular_diseases/"

# Verificando detalhes da tabela criada

In [0]:
%sql
select * from db_hospital.cardiovascular_diseases

In [0]:
%sql
desc detail db_hospital.cardiovascular_diseases

In [0]:
%sql
desc detail delta.`/hospital/rw/cardiovascular_diseases/`

# Analisando dados

In [0]:
df_cardio = spark.table("db_hospital.cardiovascular_diseases")

In [0]:
df_cardio.count()

In [0]:
df_cardio.show(10, False)

In [0]:
df_cardio.display()

# Verificando classificação de saúde cardiovascular
Pesquisa de como as pessoas classificam a sua saúde do coração

In [0]:
df_cardio.groupby("General_Health", "Sex").count().display()

# Verificando a quantidade de pessoas que tiveram frequência no hospital

In [0]:
df_cardio.groupBy("Checkup", "Age_Category").count().distinct().display()

# Verificando quem faz atividade fisica

In [0]:
df_cardio.groupBy("Exercise", "Sex").count().display()

# Verificando quem teve doença cardíaca coronária ou infarto do miocárdio

In [0]:
df_cardio.groupBy("Heart_Disease", "Sex").count().display()

In [0]:
df_cardio.groupBy("Heart_Disease", "Age_Category").count().display()